In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def pickUp(month,day,hour):
    df2 = pd.DataFrame()
    df = pd.read_csv("2020"+month+"/2020"+month+day+"/2020"+month+day+"-"+hour+".csv")
    df = df.sort_values(["VehicleID","timeR"])
    df.drop(columns=["Unnamed: 0"])
    UniqueID = df['VehicleID'].unique()
    a = df.groupby(["VehicleID"])
    for i in UniqueID:
        b = a.get_group(i)
        c = b[(b['for_hire_light'].shift(1) == 1) & (b['for_hire_light'] == 0)]  
        if not c.empty:
            c["timeDiff"] = b["timeR"].shift(1)
            df2 = pd.concat([df2,c])
        
months = ['01']
hours = ['00','01','02','03','04','05','06','07','08','09']
days = ['01']
for month in months:
    if month == '01':
        for day in days:
            for hour in hours:    
                pickUp(month,day,hour)
            print(day+'-'+month)

In [ ]:
df = pd.read_csv("202001/20200101/20200101-07.csv")
df = df.sort_values(["VehicleID","timeR"])
df.drop(columns=["Unnamed: 0"])
UniqueID = df['VehicleID'].unique()
df2 = pd.DataFrame()
a = df.groupby(["VehicleID"])
for i in UniqueID:
    b = a.get_group(i)
    c = b[(b['for_hire_light'].shift(1) == 1) & (b['for_hire_light'] == 0)]  
    if not c.empty:
        c["timeDiff"] = b["timeR"].shift(1)
        df2 = pd.concat([df2,c])
        
df2 = df2.reset_index(drop=True)
df2
#1->0 = pick / 0->1 = drop

In [ ]:
df2['hour'] = df2['timestamp'].str[11:13]
df2['day'] = df2['timestamp'].str[8:10]
df2

In [ ]:
df3 = df2[["lat","lon"]]
df3

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.01, min_samples=16)

In [ ]:
cluster = dbscan.fit_predict(df3)
cluster

In [ ]:
df4 =  pd.DataFrame(cluster,columns=["clusterID"])
df4

In [ ]:
df5 = pd.DataFrame.join(df3,df4)
df5 = df5.assign(counts=1)
df5 = df5.join(df2["day"])
df5 = df5.join(df2["hour"])
df5
df5 = df5[df5.clusterID != -1]

In [ ]:
df5 = df5.groupby(['clusterID']).agg({"lat":"mean","lon":"mean","counts":"sum"})
df5

- มีจุดไหนบ้างที่  taxi วิ่งจอดได้คนเลย
- มีจุดไหนบ้างที่ taxi จอดรอ ไม่เกิน 10 นาที
- มีจุดไหนบ้างที่  taxi จอดรอ 10-30 นาที
- มีจุดไหนบ้างที่ taxi จอด 10 นาทีแล้วไม่ได้คน จึงออกไป
- แบ่งตามวัน และช่วงเวลา
- ตัดเอาปั๊มน้ำมันออก